In [1]:
!pip install datasets transformers seqeval
!pip install accelerate -U
!pip install datasets -q
!pip install transformers -q
!pip install git+https://github.com/huggingface/evaluate.git
!pip install huggingface_hub -q
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 5.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=f840549704ac2cf5f963d0d0c855f281abbc25abbe21591004d20a80d5b956ef
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub

In [15]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np
import datasets
from evaluate import evaluator
from datasets import load_dataset
from transformers import pipeline
import torch

In [3]:
dataset = load_dataset("conll2003")
test_dataset = load_dataset("conll2003", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [5]:
label_names = dataset["train"].features["ner_tags"].feature.names
labels1 = {i: label for i, label in enumerate(label_names)}
ids = {j: id for id, j in labels1.items()}
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_names), label2id=labels1, id2label=ids)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased", padding='max_length', truncation=True, return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
def tokenize(data, label_all_tokens=True):
    input_tokens = tokenizer(data["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(data["ner_tags"]):
        word_id = input_tokens.word_ids(batch_index=i)
        prev_wrdidx = None
        id_label = []
        for word_idx in word_id:
            if word_idx is None:
                id_label.append(-100)
            elif word_idx != prev_wrdidx:
                id_label.append(label[word_idx])
            else:
                id_label.append(label[word_idx] if label_all_tokens else -100)
            prev_wrdidx = word_idx
        labels.append(id_label)
    input_tokens["labels"] = labels
    return input_tokens

tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForTokenClassification(tokenizer)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [9]:
batch_size = 16
epochs = 5
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.170600,0.063533
2,0.037900,0.061661
3,0.018000,0.061941
4,0.009000,0.066308
5,0.005100,0.069828


TrainOutput(global_step=4390, training_loss=0.03950539840924169, metrics={'train_runtime': 194.7713, 'train_samples_per_second': 360.448, 'train_steps_per_second': 22.539, 'total_flos': 851243264256672.0, 'train_loss': 0.03950539840924169, 'epoch': 5.0})

In [10]:
model=model.eval()

In [19]:
text = tokenizer(["USA is place for many international students to study. I am in class of university at buffalo"], padding = "max_length", truncation=True)
output = model(torch.tensor(text["input_ids"]).cuda(), attention_mask=torch.tensor(text["attention_mask"]).cuda())[0].argmax(dim=2)
for _, tok in enumerate(text['input_ids'][0]):
 if tok:
  print(tokenizer.decode(tok), ":", label_names[output[0][_].item()])

[CLS] : O
usa : B-LOC
is : O
place : O
for : O
many : O
international : O
students : O
to : O
study : O
. : O
i : O
am : O
in : O
class : O
of : O
university : B-ORG
at : I-ORG
buffalo : I-ORG
[SEP] : O


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("nlpproject/NER_distilBERT")

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nlpproject/NER_distilBERT/commit/43d0389a762ff687b48d5005b09c883302773b8d', commit_message='Upload DistilBertForTokenClassification', commit_description='', oid='43d0389a762ff687b48d5005b09c883302773b8d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("nlpproject/NER_distilBERT")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nlpproject/NER_distilBERT/commit/8ec307dd47f2c5e6368dcee65775edb4b18d166a', commit_message='Upload tokenizer', commit_description='', oid='8ec307dd47f2c5e6368dcee65775edb4b18d166a', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
classification_evaluator = evaluator("token-classification")

In [12]:
classification_pipeline = pipeline("token-classification", model="nlpproject/NER_distilBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [13]:
evaluation_results = classification_evaluator.compute(
    model_or_pipeline=classification_pipeline,
    data=test_dataset,
    metric="seqeval",
    input_column='tokens',
    label_column='ner_tags'
)

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
evaluation_results

{'LOC': {'precision': 0.910360884749709,
  'recall': 0.9376498800959233,
  'f1': 0.9238038984051979,
  'number': 1668},
 'MISC': {'precision': 0.747051114023591,
  'recall': 0.811965811965812,
  'f1': 0.7781569965870306,
  'number': 702},
 'ORG': {'precision': 0.8778857837181046,
  'recall': 0.8699578567128236,
  'f1': 0.8739038403386755,
  'number': 1661},
 'PER': {'precision': 0.974375,
  'recall': 0.9641311069882499,
  'f1': 0.9692259869443581,
  'number': 1617},
 'overall_precision': 0.8971538327221931,
 'overall_recall': 0.9097025495750708,
 'overall_f1': 0.9033846153846153,
 'overall_accuracy': 0.9800796812749004,
 'total_time_in_seconds': 233.34840759100007,
 'samples_per_second': 14.797615443994045,
 'latency_in_seconds': 0.06757845571705764}

In [ ]:
evaluation_results

{'LOC': {'precision': 0.910360884749709,
  'recall': 0.9376498800959233,
  'f1': 0.9238038984051979,
  'number': 1668},
 'MISC': {'precision': 0.747051114023591,
  'recall': 0.811965811965812,
  'f1': 0.7781569965870306,
  'number': 702},
 'ORG': {'precision': 0.8778857837181046,
  'recall': 0.8699578567128236,
  'f1': 0.8739038403386755,
  'number': 1661},
 'PER': {'precision': 0.974375,
  'recall': 0.9641311069882499,
  'f1': 0.9692259869443581,
  'number': 1617},
 'overall_precision': 0.8971538327221931,
 'overall_recall': 0.9097025495750708,
 'overall_f1': 0.9033846153846153,
 'overall_accuracy': 0.9800796812749004,
 'total_time_in_seconds': 241.74975799999993,
 'samples_per_second': 14.283364866905062,
 'latency_in_seconds': 0.0700115140457573}